Start by reading the table

In [0]:
df = spark.read.table("bikes.bronze.crm_prd_info")

Import the libraries

In [0]:
from pyspark.sql.functions import col, trim, when
from pyspark.sql.types import StringType

triming the columns

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

Fixing the numric null values and rename columns names

In [0]:
# clean fun
df_clean = (
    df
    
    # Handle NULL cost
    .withColumn(
        "prd_cost",
        when(col("prd_cost").isNull(), 0)
        .otherwise(col("prd_cost"))
    )
    
    # Rename columns
    .withColumnRenamed("prd_id", "product_id")
    .withColumnRenamed("prd_key", "product_key")
    .withColumnRenamed("prd_nm", "product_name")
    .withColumnRenamed("prd_cost", "product_cost")
    .withColumnRenamed("prd_line", "product_line")
    .withColumnRenamed("prd_start_dt", "start_date")
    .withColumnRenamed("prd_end_dt", "end_date")
)

df = df_clean

save the silver table

In [0]:
df.write.mode("overwrite").saveAsTable("bikes.silver.crm_product_info")